In [1]:
import requests, os, re
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner','parser'])
from collections import defaultdict
from gensim.models import Word2Vec
nlp.max_length=5000000
import re

In [8]:
res = requests.get("https://www.unt.edu/sitemap.xml")

In [9]:
df = pd.read_xml(res.text)

In [10]:
df.head()

,loc,changefreq,priority,lastmod
0,https://www.unt.edu/,daily,1.0,None
1,https://www.unt.edu/university-of-north-texas,yearly,1.0,2022-09-22T14:57Z
2,https://www.unt.edu/ada,never,NaN,2021-08-10T15:22Z
3,https://www.unt.edu/still-time-to-register,never,NaN,2019-01-11T21:36Z
4,https://www.unt.edu/story/good-start-associate...,never,NaN,2019-08-19T20:25Z


In [16]:
for i,j in tqdm(df.iterrows()):
    res = requests.get(j[0])
    try:
        if res.status_code == 200:
            with open(fr"C:\Users\shaik\US_Guy\Information_Retrieval\HTML_pages\{i}.html",'w',encoding='utf-8') as f:
                f.write(res.text)
        else:
            print("Error Urls",j[0])
    except:
        print("Exception Occured", j[0])

1429it [46:08,  1.94s/it]


In [11]:
file_data = []
for file in tqdm(os.listdir("HTML_pages/")):
    with open(f"HTML_pages/{file}",'r',encoding='utf-8') as f:
        data = f.read()
    soup = BeautifulSoup(data)
    name = int(file.split('.')[0])
    data = {"file_name": name,
            "title":soup.find('title').text,
            'text': soup.text.replace('|','').replace('\t','').replace('\n\n', '')\
    .replace('\n',' ').replace('  ','').encode("ascii", "ignore").decode('utf-8').strip(),
           'url':df.iloc[name][0]}
    file_data.append(data)

100%|██████████████████████████████████████████████████████████████████████████████| 1429/1429 [02:33<00:00,  9.33it/s]


In [12]:
df1 = pd.DataFrame(file_data)
df1 = df1.sort_values(['file_name'])
df1.reset_index(drop = True,inplace=True)
df1.to_excel("unt.xlsx", index = False)

In [27]:
df1 = pd.read_excel("unt.xlsx")

#### Second level of scraping urls

In [2]:
file_data = []
for file in tqdm(os.listdir("HTML_pages/")):
    with open(f"HTML_pages/{file}",'r',encoding='utf-8') as f:
        data = f.read()
    soup = BeautifulSoup(data)
    data = [url.get('href') for url in soup.find_all(['a', 'div'], {'href': re.compile(r'https?:')}) if 'www.unt.edu' not in url.get('href')]
    file_data.extend(data)

100%|██████████████████████████████████████████████████████████████████████████████| 1429/1429 [03:45<00:00,  6.33it/s]


In [3]:
file_data  = list(set(file_data))

In [4]:
filter_urls = [url for url in file_data if not re.search('teams\.microsoft|na01|linkedin|nam04|link\.email\.dynect|twitter|google|yahoo|facebook|maps|calender|flickr',url)]

In [5]:
len(filter_urls)

3343

In [26]:
df = pd.DataFrame({'urls':filter_urls})

In [27]:
df['docname'] = ""

In [28]:
# df.loc[0]['docname'] = "uamr"

In [29]:
df.head()

,urls,docname
0,https://news.unt.edu/news-releases/unt-defies-...,
1,http://music.unt.edu,
2,http://studentaffairs.unt.edu/access-mentoring...,
3,https://journalism.unt.edu/your-story/scholars...,
4,https://my.unt.edu/psp/papd01/EMPLOYEE/EMPL/h/...,


In [30]:
df.to_excel("firstlevel.xlsx", index = None)

In [3]:
df = pd.read_excel("firstlevel.xlsx")

In [3]:
df[2500:]

,urls,docname
2500,https://interactive.wfaa.com/pdfs/PublicFileRe...,NaN
2501,http://www.star-telegram.com/news/politics-gov...,NaN
2502,https://www.spektrum.de/news/wie-der-ginkgobau...,NaN
2503,http://edmastersonline.unt.edu/,NaN
2504,https://t.e2ma.net/message/gqc9gg/oktw44h,NaN
...,...,...
3338,https://www.psychologytoday.com/us/blog/my-lif...,NaN
3339,http://studentaffairs.unt.edu/student-activiti...,NaN
3340,http://anthropology.unt.edu/graduate-programs-...,NaN
3341,https://idea.unt.edu/multicultural-center,NaN


In [5]:
for i,j in tqdm(df[2500:].iterrows()):
    try:
        res = requests.get(j[0],timeout=5)
        if res.status_code == 200:
            with open(fr"C:\Users\shaik\US_Guy\Information_Retrieval\TEXT\{i}.txt",'w',encoding='utf-8') as f:
                f.write(res.text)
        else:
            pass
    except:
        pass

843it [41:21,  2.94s/it]


In [14]:
file_data = []
for file in tqdm(os.listdir("TEXT/")):
    with open(f"TEXT/{file}",'r',encoding='utf-8') as f:
        data = f.read()
    soup = BeautifulSoup(data)
    name = int(file.split('.')[0])
    data = {"file_name": name,
            "title":soup.find('title').text if soup.find('title') else "None",
            'text': soup.text.replace('|','').replace('\t','').replace('\n\n', '')\
    .replace('\n',' ').replace('  ','').encode("ascii", "ignore").decode('utf-8').strip(),
           'url':df.iloc[name][0]}
    file_data.append(data)

100%|██████████████████████████████████████████████████████████████████████████████| 2798/2798 [10:53<00:00,  4.28it/s]


In [15]:
df1 = pd.DataFrame(file_data)

In [16]:
df1.head()

,file_name,title,text,url
0,0,"UNT defies national trend, grows by more than ...","UNT defies national trend, grows by more than ...",https://news.unt.edu/news-releases/unt-defies-...
1,1,UNT College of Music: Home,UNT College of Music: Home Skip to main conten...,http://music.unt.edu
2,10,"Program: Performance, MM - University of North...","Program: Performance, MM - University of North...",http://catalog.unt.edu/preview_program.php?cat...
3,100,Denton Community Market,Denton Community MarketHome About Us Vendors B...,http://www.dentoncommunitymarket.com/
4,1001,Bridge,Bridge,https://unt.bridgeapp.com/learner/training/1c0...


In [19]:
df2 = pd.read_excel("unt.xlsx")

In [20]:
df1.shape

(2798, 4)

In [21]:
df2.shape

(1429, 4)

In [22]:
final_df = pd.concat([df1, df2])

In [23]:
final_df.shape

(4227, 4)

In [25]:
final_df.reset_index(drop=True, inplace=True)

In [26]:
final_df.to_excel("final_unt.xlsx", index=None)